In [1]:
"""
Notebook 12: Final Dashboard Report (Interactive Executive Summary)
==================================================================
Delivers:
 - Interactive Plotly charts (performance, drawdown, risk)
 - Key findings with markdown summaries
"""

import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
import pickle

import warnings
warnings.filterwarnings('ignore')

# File paths (adjust as needed)
project_root = Path.cwd().parent
results_metrics_dir = project_root / 'results' / 'metrics'
results_figures_dir = project_root / 'results' / 'figures' / 'final_dashboard'

results_figures_dir.mkdir(parents=True, exist_ok=True)

# Load all results
with open(results_metrics_dir / 'backtest_all_results.pkl', 'rb') as f:
    all_results = pickle.load(f)
comparison_df = pd.read_csv(results_metrics_dir / 'backtest_comparison_2023_2024.csv', index_col=0)

# Color mapping for consistency
color_map = {
    'SPY Benchmark': 'red',
    'Equal-Weighted': 'blue',
    'HRP': 'green',
    'Black-Litterman': 'orange',
    'Constrained-Optimal': 'purple',
    'RL-PPO': 'teal'
}

strategies = list(all_results.keys())

print("✅ Data loaded, ready for Plotly dashboard.")


✅ Data loaded, ready for Plotly dashboard.


In [2]:
"""
Summarize headline stats for executive summary.
"""
# "RL agent achieved X% annualized return vs Y% for S&P 500"
rl_ret = comparison_df.loc['RL-PPO', 'Annual Return']
spy_ret = comparison_df.loc['SPY Benchmark', 'Annual Return']
ret_text = f"RL agent achieved {rl_ret:.2%} annualized return vs {spy_ret:.2%} for S&P 500"

# "Sharpe ratio improved by Z% over HRP baseline"
rl_sharpe = comparison_df.loc['RL-PPO', 'Sharpe Ratio']
hrp_sharpe = comparison_df.loc['HRP', 'Sharpe Ratio']
if hrp_sharpe != 0:
    sharpe_pct = (rl_sharpe - hrp_sharpe) / abs(hrp_sharpe) * 100
else:
    sharpe_pct = np.nan
sharpe_text = f"Sharpe ratio improved by {sharpe_pct:.1f}% over HRP baseline"

# "Maximum drawdown reduced from A% to B%"
hrp_dd = comparison_df.loc['HRP', 'Max Drawdown']
rl_dd = comparison_df.loc['RL-PPO', 'Max Drawdown']
dd_text = f"Maximum drawdown reduced from {hrp_dd:.1%} (HRP) to {rl_dd:.1%} (RL agent)"

# Export for dashboard
summary_stats = {
    "RL Return": rl_ret, 
    "SPY Return": spy_ret,
    "Sharpe RL": rl_sharpe,
    "Sharpe HRP": hrp_sharpe,
    "Sharpe % Improve RL vs HRP": sharpe_pct,
    "Drawdown HRP": hrp_dd,
    "Drawdown RL": rl_dd
}

headline_list = [
    f"**{ret_text}**",
    f"**{sharpe_text}**",
    f"**{dd_text}**"
]

for line in headline_list:
    print(line)


**RL agent achieved 30.64% annualized return vs 25.64% for S&P 500**
**Sharpe ratio improved by 50.5% over HRP baseline**
**Maximum drawdown reduced from -7.7% (HRP) to -11.0% (RL agent)**


In [3]:
"""
Interactive cumulative returns (Plotly)
"""
fig = go.Figure()
for stg in strategies:
    returns = all_results[stg]['returns']
    cumulative = (1 + returns).cumprod()
    fig.add_trace(
        go.Scatter(
            x=cumulative.index, y=cumulative.values,
            mode='lines',
            name=stg,
            line=dict(color=color_map[stg], width=3)
        )
    )

fig.update_layout(
    title='Cumulative Returns: All Strategies (2023-2024)',
    xaxis_title='Date', yaxis_title='Portfolio Multiplier',
    template='plotly_white',
    legend_title='Strategy',
    font=dict(size=12),
    hovermode='x unified',
    width=950, height=450
)

fig.show()
# Optionally, save as HTML
fig.write_html(results_figures_dir / 'cumulative_returns.html')


In [4]:
"""
Table, Sharpe, Drawdown Comparison (Plotly bar)
"""
import plotly.figure_factory as ff

table_values = comparison_df[['Annual Return','Sharpe Ratio','Max Drawdown']].sort_values('Sharpe Ratio', ascending=False)
table_values_display = table_values.copy()
table_values_display['Annual Return'] = (table_values_display['Annual Return']*100).round(2)
table_values_display['Max Drawdown'] = (table_values_display['Max Drawdown']*100).round(2)

fig_table = ff.create_table(table_values_display.reset_index())
fig_table.update_layout(title='Strategy Key Metrics')

fig_table.show()
fig_table.write_html(results_figures_dir / 'metrics_table.html')

# Sharpe comparison
fig_sharpe = px.bar(
    comparison_df.reset_index().sort_values('Sharpe Ratio'),
    x='Strategy', y='Sharpe Ratio',
    color='Strategy', color_discrete_map=color_map,
    title='Risk-adjusted Return (Sharpe Ratio, 2023-2024)')
fig_sharpe.show()
fig_sharpe.write_html(results_figures_dir / 'sharpe_bar.html')

# Drawdown
fig_dd = px.bar(
    comparison_df.reset_index().sort_values('Max Drawdown'),
    x='Strategy', y='Max Drawdown',
    color='Strategy', color_discrete_map=color_map,
    title='Maximum Drawdown (Lower is Better)')
fig_dd.show()
fig_dd.write_html(results_figures_dir / 'drawdown_bar.html')


In [5]:
from IPython.display import display, Markdown

md = f'''
# Executive Summary

## Key Findings
- {headline_list[0]}
- {headline_list[1]}
- {headline_list[2]}

## Portfolio Comparison
- RL agent (PPO): **Sharpe {rl_sharpe:.2f}**, **Annualized Return {rl_ret:.2%}**, **Max Drawdown {rl_dd:.1%}**
- HRP: **Sharpe {hrp_sharpe:.2f}**, **Annualized Return {comparison_df.loc['HRP', 'Annual Return']:.2%}**, **Max Drawdown {hrp_dd:.1%}**
- S&P 500 benchmark: **Annualized Return {spy_ret:.2%}**

## Interactive Charts
Open:
- `cumulative_returns.html`: Cumulative growth, all strategies
- `metrics_table.html`: Summary table (return, Sharpe, drawdown)
- `sharpe_bar.html`: Sharpe ratio bars
- `drawdown_bar.html`: Max drawdown bars

All files are saved in: `{results_figures_dir.absolute()}`  
'''

display(Markdown(md))

# Save as HTML for managers/stakeholders:
with open(results_figures_dir / 'executive_summary.html', 'w') as f:
    f.write(md.replace('\n', '<br>'))

print("✅ All executive summary files generated.")



# Executive Summary

## Key Findings
- **RL agent achieved 30.64% annualized return vs 25.64% for S&P 500**
- **Sharpe ratio improved by 50.5% over HRP baseline**
- **Maximum drawdown reduced from -7.7% (HRP) to -11.0% (RL agent)**

## Portfolio Comparison
- RL agent (PPO): **Sharpe 1.82**, **Annualized Return 30.64%**, **Max Drawdown -11.0%**
- HRP: **Sharpe 1.21**, **Annualized Return 11.63%**, **Max Drawdown -7.7%**
- S&P 500 benchmark: **Annualized Return 25.64%**

## Interactive Charts
Open:
- `cumulative_returns.html`: Cumulative growth, all strategies
- `metrics_table.html`: Summary table (return, Sharpe, drawdown)
- `sharpe_bar.html`: Sharpe ratio bars
- `drawdown_bar.html`: Max drawdown bars

All files are saved in: `/Users/aryamansingh/Desktop/adaptive_portfolio_manager/results/figures/final_dashboard`  


✅ All executive summary files generated.
